<a href="https://www.kaggle.com/code/alimtharwat/loan-approval-dataset-analysis?scriptVersionId=246139495" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

Team Members and IDs: Michael Danial 13004528, Ali Tharwat 13004222,
                       Sherif Tamer 13004065

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from IPython.display import display


df = pd.read_csv('/kaggle/input/loan-approval-dataset/loan_approval_dataset.csv')

#Dataset analysis and preparation tasks
#a

#Displaying the first 12 rows of the dataset(the head)
print("First 12 rows:")
display(df.head(12))

#Displaying the last 12 rows of the dataset(the tail)
print("Last 12 rows:")
display(df.tail(12))

In [ ]:
#b
#Printing the total number of columns and the total number of rows
print("Total Rows and Columns:", df.shape)

In [ ]:
#c
#Printing the name of each feature and their corresponding data type
print("Column Names and Data Types:")
print(df.dtypes)

In [ ]:
#d
#Printing the name of the first feature
print("First Column Name:", df.columns[0])

In [ ]:
#e
#Shows non-null counts and data types of each column ,It has 5,000 rows which are indexed from 0 to 4999.It has 14 columns.
print("Dataset Summary:")
print(df.info())

In [ ]:
#f
#We have chosen the 'Age' attribute and have displayed it's distinct values.(no duplicates)
categorical_column = 'Age'
print(f"Distinct values in '{categorical_column}':", df[categorical_column].unique())

In [ ]:
#g
#Displaying the most occuring value for the 'Age' attribute which is 64.
mfv = df[categorical_column].mode()[0]
print(f"\nMost frequently occurring value in '{categorical_column}':", mfv)


In [ ]:
#h
#Note:all following functions display numeric values only
print("Statistical Summary: \n")
#Displaying the mean(average) for each attribute
print("Mean:\n\n", df.mean(numeric_only=True))
#Displaying the median value for each attribute
print("\nMedian:\n\n", df.median(numeric_only=True))
#Displaying standard deviation(a measure of how much the values in a dataset deviate(spread out) from the mean).
print("\nStandard Deviation:\n\n", df.std(numeric_only=True))
#Displaying the 25th percentile (Q1), 50th percentile (Median or Q2), and 75th percentile (Q3)
print("\nPercentiles:\n\n", df.quantile([0.25, 0.5, 0.75], numeric_only=True))

In [ ]:
#Data preparation tasks

#a
#Selecting Rows with Loan Amount more than 498000
filtered_df = df[df['Loan_Amount'] > 498000]
print("Filtered Data:\n", filtered_df)

In [ ]:
#b
if 'Marital_Status' in df.columns:
    count_S = df[df['Marital_Status'].str.startswith('S', na=False)].shape[0]
    print(f"Number of records where 'Marital Status' starts with 'S': {count_S}")
else:
    print("No 'Marital_Status' column found.")


In [ ]:
#c

# count duplicated rows
duplicates = df.duplicated().sum()
print("Total duplicate rows:", duplicates)

# remove duplicated rows
df = df.drop_duplicates()

In [ ]:
#d

#converting Credit_Score from integer to string
df['Credit_Score'] = df['Credit_Score'].astype(str)
print("Updated data types:\n", df.dtypes)


In [ ]:
#e
# grouping data by : Property_Ownership &  Marital_Status
grouped = df.groupby(['Property_Ownership', 'Marital_Status']).size()
print("Grouped Data:\n", grouped)

#some analysis for the data :
# counts range between 391 and 436 , making them a nearly uniform distribution
# no single combination dominates the dataset, and all groups are well-represented.

In [ ]:
#f
print("Missing Values:\n", df.isnull().sum())

In [ ]:
#g
for col in df.columns:
    if df[col].dtype == 'object':  # Categorical data
        df[col].fillna(df[col].mode()[0], inplace=True)
    else:  # Numerical data
        df[col].fillna(df[col].median(), inplace=True)

print("Missing values after change:\n", df.isnull().sum())


In [ ]:
#h
df['Income_Bins'] = pd.cut(df['Income'], bins=5)
print(df['Income_Bins'].value_counts())


In [ ]:
#i
max_row = df[df['Credit_Score'] == df['Credit_Score'].max()]
print("Row with maximum credit score:\n", max_row)


In [ ]:
#j
# we chose the Loan_Amount as is a critical factor in the loan approval process.
# it directly impacts the borrower's ability to repay the loan and the lender's risk assessment.

plt.figure(figsize=(6, 4))
sns.boxplot(x=df['Loan_Amount'].astype(float))
plt.title("Boxplot of Loan Amount")
plt.show()


In [ ]:
#k
plt.figure(figsize=(6, 4))
plt.hist(df['Income'], bins=30, edgecolor='black')
plt.xlabel("Applicant Income")
plt.ylabel("Frequency")
plt.title("Histogram of Applicant Income")
plt.show()

#For the Income attribute, a histogram helps visualize:
#The central tendency(where most incomes lie) , range of incomes , shape of the distribution , etc


In [ ]:
#l
plt.figure(figsize=(6, 4))
sns.scatterplot(x=df['Income'], y=df['Loan_Amount'].astype(float))
plt.xlabel("Applicant Income")
plt.ylabel("Loan Amount")
plt.title("Scatterplot of Applicant Income vs Loan Amount")
plt.show()

#The scatterplot shows a positive relationship between applicant income and loan amount
# indicating that higher-income applicants tend to request larger loans.

In [ ]:
#m
scaler = StandardScaler()
df_scaled = df.copy()
numeric_cols = df.select_dtypes(include=[np.number]).columns
df_scaled[numeric_cols] = scaler.fit_transform(df[numeric_cols])
print("Standardized Data:\n", df_scaled.head())

In [ ]:
#n
pca = PCA(n_components=2)
pca_data = pca.fit_transform(df_scaled[numeric_cols])

# Convert to DataFrame
df_pca = pd.DataFrame(pca_data, columns=['PC1', 'PC2'])

# Scatterplot before PCA
plt.figure(figsize=(6, 4))
sns.scatterplot(x=df_scaled[numeric_cols[0]], y=df_scaled[numeric_cols[1]])
plt.title("Before PCA")
plt.show()

# Scatterplot after PCA
plt.figure(figsize=(6, 4))
sns.scatterplot(x=df_pca['PC1'], y=df_pca['PC2'])
plt.title("After PCA")
plt.show()

In [ ]:
#o
plt.figure(figsize=(8, 6))
sns.heatmap(df.corr(numeric_only=True), annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Correlation Heatmap")
plt.show()


In [ ]:
#Practical analytical questions

#a
correlation_matrix = df.corr(numeric_only=True)
print("Correlation Matrix:")
display(correlation_matrix.style.background_gradient(cmap="coolwarm").format(precision=2))


# analysis :all our correlations are very weak correlations

# however Loan_Amount and Income are potential features relevant for classification

In [ ]:
#b
if 'Employment_Status' in df.columns:
    class_distribution = df['Employment_Status'].value_counts()
    print("Class Distribution:\n", class_distribution)

    # Visualizing the distribution
    plt.figure(figsize=(6, 4))
    sns.barplot(x=class_distribution.index, y=class_distribution.values)
    plt.xlabel("Employment Status")
    plt.ylabel("Count")
    plt.title("Class Distribution of Employment Status")
    plt.show()
else:
    print("Column 'Employment_Status' not found. Choose another categorical column.")

# The class distribution shows that the dataset is balanced fairly across the three employment status


In [ ]:
#c
# 1. Creating 'Income-to-Loan Ratio' (higher values might indicate better repayment capacity)
df['Income_to_Loan_Ratio'] = df['Income'] / df['Loan_Amount'].replace(0, np.nan)
# 2. Creating 'EMI' (Equated Monthly Installment) to assess financial burden
df['EMI'] = df['Loan_Amount'].replace(0, np.nan) / df['Loan_Term'].replace(0, np.nan)
new_features = ['Income_to_Loan_Ratio', 'EMI']
print("Newly Created Features:\n", df[new_features].head())
#Explanations of significance/enhancements
#Income-to-Loan Ratio: A higher ratio indicates that an applicant earns significantly more than their loan amount, suggesting lower financial risk and higher loan approval chances.
#EMI (Equated Monthly Installment): Helps assess the monthly financial burden on an applicant; lower EMI values indicate better affordability, making the applicant more likely to repay the loan successfully.
